# 도봉구 (1089)
- 작성자 : 황다은
- 작성일자 : 2020년 9월 16일
- msk와 mob가 같은 항에 뒤죽박죽 있어서 전처리하지 못함.

In [253]:
from bs4 import BeautifulSoup
import datetime as dt
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

dat = dt.datetime.today().date() # 오늘 날짜

### 1. Raw File 불러오기

In [254]:
df = pd.read_csv("200916one_shot.csv")
soup = BeautifulSoup(df['html'][5], 'html.parser')

### 2. 간단한 전처리 하기

In [255]:
rows = soup.select('div.corona_info > ul')
rows1 = soup.select('div.corona_info > div.info_detail')

### 3. 저장할 DataFrame 미리 만들기

In [256]:
pat_col = ['num', 'dat', 'pth', 'syy', 'syd', 'cnd', 'rsd', 'atn', 'dch', 'adt','age','sex']
rou_col = ['num', 'ord', 'dat', 'rgl', 'rgt', 'frm', 'exd', 'msk', 'ste', 'mob']
patient = pd.DataFrame(columns = pat_col)
route = pd.DataFrame(columns = rou_col)

### 4. 귀찮은 전처리해주기

In [257]:
for i in tqdm(range(len(rows))):
    
    title = rows[i].select('li')
    
    if '구' in title[0]:
        numindex = title[0].text.index('구')
        num = '1089' + '0' * (5- len(title[0].text[numindex+1:numindex+3])) + title[0].text[numindex+1:numindex+3]
    else:
        numindex = title[0].text.index('#')
        num = '1089' + '0' * (5- len(title[0].text[numindex+1:numindex+4])) + title[0].text[numindex+1:numindex+4]
    dch = title[1].text
    cnd = title[2].text
    pth = title[3].text
    atn = title[4].text

    if '비공개' in rows1[i].text:
        rsd = ''; adt = ''; age = ''
    else:
        rsdindex = rows1[i].text.split('○')[0].split('거주')[0].index('(') + 1
        rsd = rows1[i].text.split('○')[0].split('거주')[0][rsdindex:]
        ageindex = rows1[i].text.split('○')[0].split('거주')[1].index('대')
        age = rows1[i].text.split('○')[0].split('거주')[1][:ageindex]
        adtloc = len(rows1[i].text.split('○'))-1
        adtindex = rows1[i].text.split('○')[adtloc].split('역학조사 결과 : ')[1].index('\n')
        adt = rows1[i].text.split('○')[adtloc].split('역학조사 결과 : ')[1][:adtindex]
    syy = ''; syd = ''; sex = '';
    new_pat = pd.DataFrame([(num, dat, pth, syy, syd, cnd, rsd, atn, dch, adt, age, sex)], columns = pat_col)
    patient = pd.concat([patient, new_pat])
    
    if '비공개' in rows1[i].text:
        new_route = pd.DataFrame(columns = rou_col, data = [[num, 1, dat, '', '', '', '', '', '', '']])
        route = pd.concat([route, new_route])
    else:
        order = 0
        for x in range(1,len(rows1[i].text.split('○'))-1):
            if '소독완료' in rows1[i].text.split('○')[len(rows1[i].text.split('○'))-1]:
                ste = 1
            else:
                ste = 0
            for y in range(1, len(rows1[i].text.split('○')[x].split('- '))):
                
                exd = rows1[i].text.split('○')[x].split('- ')[0]
                exd += rows1[i].text.split('○')[x].split('- ')[y].split('|')[0]
                try:
                    rgt = rows1[i].text.split('○')[x].split('- ')[y].split('|')[1]
                except:
                    rgt = ''
                order += 1
                new_route = pd.DataFrame([(num, order, dat, '', rgt, '', exd,'',ste,'')], columns = rou_col)
                route = pd.concat([route, new_route])
                
        

        


        
    

100%|████████████████████████████████████████████████████████████████████████████████| 101/101 [00:03<00:00, 33.11it/s]


### 5. 결과 잘 나왔나 확인하기

In [258]:
patient

,num,dat,pth,syy,syd,cnd,rsd,atn,dch,adt,age,sex
0,108900163,2020-09-16,타지역 확진자 접촉자,,,09.14.(월),도봉1동,서북병원,치료중,"접촉자 5명(동거가족 2, 지인 3), 검사 진행중(5명)",50,
0,108900162,2020-09-16,안양시 확진자 접촉자,,,09.13.(일),도봉1동,태릉생활치료센터,치료중,동거가족 1명 양성,50,
0,108900161,2020-09-16,안양시 확진자 접촉자,,,09.13.(일),도봉1동,태릉생활치료센터,치료중,동거가족 1명 양성,50,
0,108900160,2020-09-16,도봉157번 확진자 접촉자,,,09.12.(토),쌍문1동,삼성생활치료센터,치료중,동거가족2명 양성,50,
0,108900159,2020-09-16,도봉157번 확진자 접촉자,,,09.12.(토),쌍문1동,삼성생활치료센터,치료중,동거가족 2명 양성,30,
...,...,...,...,...,...,...,...,...,...,...,...,...
0,108900145,2020-09-16,도봉 #65 접촉자,,,08.06.(목),,국가지정격리병상,완치,,,
0,108900144,2020-09-16,도봉 #65 접촉자,,,08.04.(화),,국가지정격리병상,완치,,,
0,108900144,2020-09-16,불명,,,08.03.(월),,국가지정격리병상,완치,,,
0,108900139,2020-09-16,해외 입국자,,,07.23.(목),,국가지정격리병상,완치,,,


In [259]:
route

,num,ord,dat,rgl,rgt,frm,exd,msk,ste,mob
0,108900163,1,2020-09-16,,자택,,9.9.(수)11:40,,1,
0,108900163,2,2020-09-16,,자택→직장,,9.9.(수)11:40-12:16,,1,
0,108900163,3,2020-09-16,,직장(창동),,9.9.(수)12:16-18:23,,1,
0,108900163,4,2020-09-16,,직장→자택,,9.9.(수)18:23-18:50,,1,
0,108900163,5,2020-09-16,,자택→직장,,9.10.(목)11:10-11:42,,1,
...,...,...,...,...,...,...,...,...,...,...
0,108900145,1,2020-09-16,,,,,,,
0,108900144,1,2020-09-16,,,,,,,
0,108900144,1,2020-09-16,,,,,,,
0,108900139,1,2020-09-16,,,,,,,
